In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/1.part'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/input'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            data = {'search': crawler_series['Part Number'],
                    'page': '1'}

            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_check = f'''https://uacparts.com/Catalog/SearchIndex?search={crawler_series['Part Number']}'''

                    resp = requests.post('https://uacparts.com/SearchAjaxList/Search',
                                         data=data,
                                         headers=get_header(),
                                         proxies=get_proxy(),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if dict_['d']['it'] == None and dict_['d']['m'] == False and dict_['d']['th'] == None and dict_['d']['p'] == None:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(dict_['d']['c'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_li = html.xpath('//li[@class="awe-li part awe-il"]')

            # = = = = = = = = = = = = = = =

            list_part_number = [li.xpath('./div[@class="label-bar"]/h2[@class="part-label"]/text()')[0] for li in list_li]

            # = = = = = = = = = = = = = = =

            array_index_part = np.where(np.array([part_number.replace(' ', '') for part_number in list_part_number]) == crawler_series['Part Number'])[0]

            # = = = = = = = = = = = = = = =

            if array_index_part.size == 1:
                index_part = array_index_part[0]
            else:
                raise

            # = = = = = = = = = = = = = = =

            dict_src, list_src = {}, [href.strip() for href in list_li[index_part].xpath('./div/div[@class="part-image"]/img[@alt="picture"]/@src')]
            for i, src in enumerate(list_src):
                dict_src[str(i)] = src

            # = = = = = = = = = = = = = = =

            dict_note, list_text = {}, list_li[index_part].xpath('./div/div[@class="part-info"]/div[@class="comments"]/text()')
            for i, text in enumerate(list_text):
                dict_note[str(i)] = text

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'Part Number': crawler_series['Part Number'],
                                          '[UAC] Part Number': list_part_number[index_part],
                                          '[UAC] Picture': '',
                                          '[UAC] Url': url_check,
                                          '[UAC] Json_Src': json.dumps(dict_src),
                                          '[UAC] Json_Note': json.dumps(dict_note)})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Number'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Number'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/22 09:41:40] - 2. https://uacparts.com/Catalog/SearchIndex?search=HA10020C

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/22 09:41:40] - 9. https://uacparts.com/Catalog/SearchIndex?search=HA10066C

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/22 09:41:40] - 3. https://uacparts.com/Catalog/SearchIndex?search=HA10021C

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/22 09:41:40] - 5. https://uacparts.com/Catalog/SearchIndex?search=HA10029C

[True] - [请求2次] - [剩余5条] - [crawler_1（50.00%）] - [2024/10/22 09:41:41] - 1. https://uacparts.com/Catalog/SearchIndex?search=HA10018C

[True] - [请求1次] - [剩余4条] - [crawler_1（50.00%）] - [2024/10/22 09:41:41] - 4. https://uacparts.com/Catalog/SearchIndex?search=HA10022C

[True] - [请求1次] - [剩余3条] - [crawler_1（50.00%）] - [2024/10/22 09:41:41] - 6. https://uacparts.com/Catalog/SearchIndex?search=HA10034C

[True] - [请求1次] - [剩余2条] - [crawler_1（50.00%）] - [2024

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3724.96it/s]

Done ~
